In [44]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/CSC420/CSC420_project-main/src'
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/CSC420/CSC420_project-main/src
arch					      infer.py	   test_out
arch_st					      __init__.py  test.py
data					      log.log	   train_out
finetuning_torchvision_models_tutorial.ipynb  main.py	   train.py
generate_lr.py				      pretrained   util


In [ ]:
# Remove problematic images (not the same size as all others)

# %cd '/content/drive/My Drive/CSC420/CSC420_project-main/src/data/pokemon/hr'
# !rm '077_f2.png' '555_f2.png' '263_f2.png' '110_f2.png' '745_f2.png' '422_f2.png' '264_f2.png' '745_f3.png'

# %cd '/content/drive/My Drive/CSC420/CSC420_project-main/src/data/pokemon/lr'
# !rm '077_f2.png' '555_f2.png' '263_f2.png' '110_f2.png' '745_f2.png' '422_f2.png' '264_f2.png' '745_f3.png'

# %cd '/content/drive/My Drive/CSC420/CSC420_project-main/src'

/content/drive/My Drive/CSC420/CSC420_project-main/src/data/pokemon/hr
/content/drive/My Drive/CSC420/CSC420_project-main/src/data/pokemon/lr
/content/drive/My Drive/CSC420/CSC420_project-main/src


In [33]:
# Split all data into train, validation, test sets

# %cd '/content/drive/My Drive/CSC420/CSC420_project-main/src/data/pokemon/lr'
# import os
# import numpy as np
# from shutil import copyfile

# train_fp, val_fp, test_fp = [], [], []
# for fp in os.listdir():
#     if os.path.isfile(fp) and os.path.splitext(fp)[1] == ".png":
#         val = np.random.random()
#         if val < 0.20:
#             test_fp.append(fp)
#         elif val < 0.40:
#             val_fp.append(fp)
#         else:
#             train_fp.append(fp)

# for folder in ["train", "val", "test"]:
#     if folder not in os.listdir():
#         os.mkdir(folder)

# base_fp = '/content/drive/My Drive/CSC420/CSC420_project-main/src/data/pokemon/lr/'
# for folder, fps in [("train/", train_fp), ("val/", val_fp), ("test/", test_fp)]:
#     for fp in fps:
#         copyfile(base_fp + fp, base_fp + folder + fp)


# %cd '/content/drive/My Drive/CSC420/CSC420_project-main/src/data/pokemon/hr'

# for folder in ["train", "val", "test"]:
#     if folder not in os.listdir():
#         os.mkdir(folder)

# base_fp = '/content/drive/My Drive/CSC420/CSC420_project-main/src/data/pokemon/hr/'
# for folder, fps in [("train/", train_fp), ("val/", val_fp), ("test/", test_fp)]:
#     for fp in fps:
#         copyfile(base_fp + fp, base_fp + folder + fp)

/content/drive/My Drive/CSC420/CSC420_project-main/src/data/pokemon/lr


In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

import argparse
import logging
logging.basicConfig(filename='./log.log')

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms

from arch.srgan_model import Generator, Discriminator
from arch.vgg19 import vgg19
from data.dataset import *
from arch.losses import TVLoss, perceptual_loss
from util import arg_util

PyTorch Version:  1.7.0+cu101
Torchvision Version:  0.8.1+cu101


In [47]:
memcache=False
batch_size=32
num_workers=0

scale=4
patch_size=24
model_res_count=16

transfer_generator_path=arg_util.path_abs("pretrained/SRResNet.pt")

feat_layer='relu5_4'
vgg_rescale_coeff=0.006
adv_coeff=1e-3
tv_loss_coeff=0.0

t_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
t_device

device(type='cuda')

In [96]:
import csv
from skimage.color import rgb2ycbcr
from skimage.metrics import peak_signal_noise_ratio
from torch.utils.data import DataLoader

# Get PSNR for train, val, or test sets
def test(generator_path=None, generator=None, mode="val", write_img=False):
    if mode == "train":
        output_path = arg_util.path_abs("train_out/")
        psnr_result_filepath = arg_util.path_abs("train_out/psnr.txt")
        lr_path = arg_util.path_abs("data/pokemon/lr/train")
        gt_path = arg_util.path_abs("data/pokemon/hr/train")
    elif mode == "test":
        output_path = arg_util.path_abs("test_out/")
        psnr_result_filepath = arg_util.path_abs("test_out/psnr.txt")
        lr_path = arg_util.path_abs("data/pokemon/lr/test")
        gt_path = arg_util.path_abs("data/pokemon/hr/test")
    else:
        output_path = arg_util.path_abs("val_out/")
        psnr_result_filepath = arg_util.path_abs("val_out/psnr.txt")
        lr_path = arg_util.path_abs("data/pokemon/lr/val")
        gt_path = arg_util.path_abs("data/pokemon/hr/val")
    psnr_result_filepath.parent.mkdir(parents=True, exist_ok=True)
    # print(lr_path, gt_path)
    memcache = False
    filename_prefix=""

    lr_gt_dataset = LowResGroundTruthDataset(lr_dir=lr_path, gt_dir=gt_path, memcache=memcache, transform=None)
    loader = DataLoader(lr_gt_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    if generator_path:
        generator = Generator(img_feat=3, n_feats=64, kernel_size=3, num_block=model_res_count, scale=scale)
        generator.load_state_dict(torch.load(generator_path, map_location=t_device))
        generator = generator.to(t_device)
    generator.eval()

    # print(f"Starting Testing.")
    logging.info(f"Starting Testing.")
    with torch.no_grad():
        psnr_data = []
        psnr_vals = []
        for batch_i, lr_gt_datum in enumerate(loader):
            img_filenames, img_lrs = lr_gt_datum['img_filename'], lr_gt_datum['img_lr'].to(t_device)
            img_hr_predictions, _ = generator(img_lrs)
            img_gts = lr_gt_datum['img_gt'].to(t_device)
            for i in range(len(img_filenames)):  # Independent of batch size.
                img_filename = img_filenames[i]

                img_gt = ((img_gts[i].cpu().numpy() + 1.) / 2.).transpose(1, 2, 0)
                img_hr_prediction = ((np.clip(img_hr_predictions[i].cpu().numpy(), -1., 1.) + 1.) / 2.).transpose(1, 2, 0)

                # Since no transforms are used, resize GT to ensure its the same as HR.
                img_gt = img_gt[:img_hr_prediction.shape[0], :img_hr_prediction.shape[1], :]

                # Calculate psnr from ycbcr comparison.
                y_hr_prediction = rgb2ycbcr(img_hr_prediction)[scale:-scale, scale:-scale, :1]
                y_gt = rgb2ycbcr(img_gt)[scale:-scale, scale:-scale, :1]
                psnr = peak_signal_noise_ratio(y_gt / 255., y_hr_prediction / 255., data_range=1.)
                psnr_data.append({'img_filepath': str(lr_gt_dataset.lr_dir / img_filename), 'psnr': psnr})
                logging.info(f"{img_filename}: psnr={psnr}")
                # print(f"{img_filename}: psnr={psnr}")
                psnr_vals.append(psnr)
                if write_img:
                    result = Image.fromarray((img_hr_prediction * 255.).astype(np.uint8))
                    result.save(output_path / f"pred_{img_filename}")
                    logging.info(f"Inference Output: {output_path / f'pred_{img_filename}'}")
        # print(f"Average PSNR: {sum(psnr_vals)/len(psnr_vals)}")

        if psnr_result_filepath:
            with open(psnr_result_filepath, 'w') as fp_psrn_results:
                writer = csv.DictWriter(fp_psrn_results, fieldnames=['img_filepath', 'psnr'])
                writer.writeheader()
                writer.writerows(psnr_data)

    generator.train()
    return sum(psnr_vals)/len(psnr_vals)

In [97]:
test(transfer_generator_path, mode="train")

Average PSNR: 20.159482897498478


20.159482897498478

In [98]:
# Load Training Data
generator_path_out = arg_util.path_abs("train_out/SRGAN_g.pt")
generator_path_out.parent.mkdir(parents=True, exist_ok=True)

discriminator_path_out = arg_util.path_abs("train_out/SRGAN_d.pt")
discriminator_path_out.parent.mkdir(parents=True, exist_ok=True)

checkpoint_dir = arg_util.path_abs("train_out/")
checkpoint_dir.mkdir(parents=True, exist_ok=True)

gt_path = arg_util.path_abs("data/pokemon/hr/train/")
lr_path = arg_util.path_abs("data/pokemon/lr/train/")

lr_gt_dataset = LowResGroundTruthDataset(
    lr_dir=lr_path, gt_dir=gt_path, memcache=memcache,
    transform=transforms.Compose([
        Crop_LR_GT_PairTransform(scale, patch_size),
        Random_LR_GT_AugmentationTransform()
    ])
)

loader = DataLoader(lr_gt_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
generator = Generator(img_feat=3, n_feats=64, kernel_size=3, num_block=model_res_count, scale=scale)
if transfer_generator_path:
    generator.load_state_dict(torch.load(transfer_generator_path, map_location=t_device))
    logging.info(f"Loaded pre-trained model: {transfer_generator_path}")
    print(f"Loaded pre-trained model: {transfer_generator_path}")
generator = generator.to(t_device)
_ = generator.train()

In [100]:
# Freeze all layer weights except the last few, needs testing
for param in generator.parameters():
    param.requires_grad = False
    
for param in generator.last_conv.body.parameters():
    param.requires_grad = True

for param in generator.tail.parameters():
    param.requires_grad = True

# for param in generator.conv02.parameters():
#     param.requires_grad = True

# for param in generator.body[15].parameters():
#     param.requires_grad = True

In [ ]:
def train(init_lr=1e-4, pre_train_epoch=100):
    L2_MSE_loss = nn.MSELoss()
    g_optim = optim.Adam(generator.parameters(), lr=init_lr)
    g_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(g_optim, mode="max", factor=0.1, patience=10, cooldown=1, verbose=True)

    checkpoint_modulo = (pre_train_epoch // 3) or pre_train_epoch
    for pre_epoch in range(1, pre_train_epoch + 1):
        logging.info(f"Pre-train Epoch [{pre_epoch}]: running.")
        for batch_i, lr_gt_datum in enumerate(loader):
            img_lr, img_gt = lr_gt_datum['img_lr'].to(t_device), lr_gt_datum['img_gt'].to(t_device)
            img_hr_prediction, _ = generator(img_lr)
            loss = L2_MSE_loss(img_hr_prediction, img_gt)
            g_optim.zero_grad()
            loss.backward()
            g_optim.step()

        # Log epoch statistics.
        logging.info(f"Pre-train Epoch [{pre_epoch}]: loss={loss.item()}")
        print(f"Pre-train Epoch [{pre_epoch}]: training loss={loss.item()}")
        # _ = test(generator=generator, mode="train", write_img=False)
        val_psnr = test(generator=generator, mode="val", write_img=False)

        print(f"Pre-train Epoch [{pre_epoch}]: validation PSNR={val_psnr}")
        g_scheduler.step(val_psnr)

        if pre_epoch % checkpoint_modulo == 0:
            checkpoint_filepath = (checkpoint_dir / f'pre_trained_model_{pre_epoch}.pt').absolute()
            torch.save(generator.state_dict(),  checkpoint_filepath)
            logging.info(f"Pre-train Epoch [{pre_epoch}]: saved model checkpoint: {checkpoint_filepath}")

In [101]:
generator_path_out = arg_util.path_abs("train_out/SRGAN_g_last_conv.pt")
generator_path_out.parent.mkdir(parents=True, exist_ok=True)
torch.save(generator.state_dict(), generator_path_out)

In [ ]:
%%timeit
# Pre-train using L2 loss
pre_train_epoch = 100
checkpoint_modulo = (pre_train_epoch // 3) or pre_train_epoch
for pre_epoch in range(1, pre_train_epoch + 1):
    logging.info(f"Pre-train Epoch [{pre_epoch}]: running.")
    for batch_i, lr_gt_datum in enumerate(loader):
        img_lr, img_gt = lr_gt_datum['img_lr'].to(t_device), lr_gt_datum['img_gt'].to(t_device)
        img_hr_prediction, _ = generator(img_lr)
        loss = L2_MSE_loss(img_hr_prediction, img_gt)
        g_optim.zero_grad()
        loss.backward()
        g_optim.step()

    # Log epoch statistics.
    logging.info(f"Pre-train Epoch [{pre_epoch}]: loss={loss.item()}")
    print(f"Pre-train Epoch [{pre_epoch}]: training loss={loss.item()}")
    # _ = test(generator=generator, mode="train", write_img=False)
    val_psnr = test(generator=generator, mode="val", write_img=False)

    print(f"Pre-train Epoch [{pre_epoch}]: validation PSNR={val_psnr}")
    g_scheduler.step(val_psnr)

    if pre_epoch % checkpoint_modulo == 0:
        checkpoint_filepath = (checkpoint_dir / f'pre_trained_model_{pre_epoch}.pt').absolute()
        torch.save(generator.state_dict(),  checkpoint_filepath)
        logging.info(f"Pre-train Epoch [{pre_epoch}]: saved model checkpoint: {checkpoint_filepath}")

Pre-train Epoch [1]: running.
Pre-train Epoch [1]: loss=0.0905727669596672
Average PSNR: 20.48694627317539
Average PSNR: 20.583544995220155
Pre-train Epoch [2]: running.
Pre-train Epoch [2]: loss=0.06489412486553192
Average PSNR: 20.624586816973398
Average PSNR: 20.723522937269646
Pre-train Epoch [3]: running.
Pre-train Epoch [3]: loss=0.06203813850879669
Average PSNR: 20.74324555161233
Average PSNR: 20.842975283628395
Pre-train Epoch [4]: running.
Pre-train Epoch [4]: loss=0.06971778720617294
Average PSNR: 20.885333229521894
Average PSNR: 20.986590571456574
Pre-train Epoch [5]: running.
Pre-train Epoch [5]: loss=0.05519309267401695
Average PSNR: 20.96703452886423
Average PSNR: 21.069493667145796
Pre-train Epoch [6]: running.
Pre-train Epoch [6]: loss=0.05596834048628807
Average PSNR: 21.052831640707986
Average PSNR: 21.156880805168218
Pre-train Epoch [7]: running.
Pre-train Epoch [7]: loss=0.06602142006158829
Average PSNR: 21.144252978052357
Average PSNR: 21.24914421337722
Pre-train E

In [ ]:
test(generator=generator, mode="test", write_img=True)

In [ ]:
# Below is Work In Progress

In [ ]:



# Train using perceptual & adversarial loss
if adversarial_train_epoch > 0:
    logging.info(f"Training using Adversarial loss for {adversarial_train_epoch} epochs.")

    # Set-up adversarial loss VGG network.
    vgg_net = vgg19().to(t_device)
    vgg_net = vgg_net.eval()

    discriminator = Discriminator(patch_size=patch_size * scale)
    discriminator = discriminator.to(t_device)
    discriminator.train()

    d_optim = optim.Adam(discriminator.parameters(), lr=1e-4)
    scheduler = optim.lr_scheduler.StepLR(g_optim, step_size=2000, gamma=0.1)

    VGG_loss = perceptual_loss(vgg_net)
    cross_ent = nn.BCELoss()
    tv_loss = TVLoss()
    base_real_label = torch.ones((batch_size, 1)).to(t_device)
    base_fake_label = torch.zeros((batch_size, 1)).to(t_device)

    torch.autograd.set_detect_anomaly(True)
    checkpoint_modulo = (adversarial_train_epoch // 10) or adversarial_train_epoch
    for epoch in range(1, adversarial_train_epoch + 1):
        logging.info(f"Epoch [{epoch}]: running.")

        d_optim.step()
        g_optim.step()
        scheduler.step()
        for batch_i, lr_gt_datum in enumerate(loader):
            img_lr, img_gt = lr_gt_datum['img_lr'].to(t_device), lr_gt_datum['img_gt'].to(t_device)
            img_hr_prediction, _ = generator(img_lr)

            # Train Discriminator
            fake_prob = discriminator(img_hr_prediction)
            real_prob = discriminator(img_gt)

            # Avoid mismatched label and probability length in case where batch is remainder of data, but not
            # a perfect fit.
            real_label = base_real_label
            fake_label = base_fake_label
            if len(base_real_label) != len(real_prob):
                real_label = torch.ones((len(real_prob), 1)).to(t_device)
                fake_label = torch.zeros((len(real_prob), 1)).to(t_device)

            d_loss_real = cross_ent(real_prob, real_label)
            d_loss_fake = cross_ent(fake_prob, fake_label)

            d_loss = d_loss_real + d_loss_fake

            # Back-propagate Discriminator
            g_optim.zero_grad()
            d_optim.zero_grad()
            d_loss.backward()
            d_optim.step()

            # Train Generator
            img_hr_prediction, _ = generator(img_lr)
            fake_prob = discriminator(img_hr_prediction)

            l2_loss = L2_MSE_loss(img_hr_prediction, img_gt)
            percep_loss, hr_feat, sr_feat = VGG_loss((img_gt + 1.0) / 2.0, (img_hr_prediction + 1.0) / 2.0, layer=feat_layer)
            percep_loss = vgg_rescale_coeff * percep_loss
            adversarial_loss = adv_coeff * cross_ent(fake_prob, real_label)
            total_variance_loss = tv_loss_coeff * tv_loss(vgg_rescale_coeff * (hr_feat - sr_feat) ** 2)
            g_loss = percep_loss + adversarial_loss + total_variance_loss + l2_loss

            # Back-propagate Generator
            g_optim.zero_grad()
            d_optim.zero_grad()
            g_loss.backward()
            g_optim.step()

        # Log epoch statistics.
        logging.info(f"Epoch [{epoch}]: g_loss={g_loss.item()} d_loss={d_loss.item()}")
        if epoch % checkpoint_modulo == 0:
            g_checkpoint_filepath = (checkpoint_dir / f'SRGAN_g_{epoch}.pt').absolute()
            d_checkpoint_filepath = (checkpoint_dir / f'SRGAN_d_{epoch}.pt').absolute()
            torch.save(generator.state_dict(),  g_checkpoint_filepath)
            torch.save(discriminator.state_dict(), d_checkpoint_filepath)
            logging.info(f"Pre-train Epoch [{epoch}]: saved model checkpoints: {g_checkpoint_filepath}, {d_checkpoint_filepath}")
    if discriminator_path_out:
        torch.save(discriminator.state_dict(), discriminator_path_out)
torch.save(generator.state_dict(), generator_path_out)





ls: cannot access '/content/drive/My': No such file or directory
ls: cannot access 'Drive/CSC420/CSC420_project-main/src/pretrained/': No such file or directory


KeyboardInterrupt: ignored